In [1]:
import torch 
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
import pandas as pd

data_path = '/kaggle/input/plant-pathology-2020-fgvc7/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [4]:
from sklearn.model_selection import train_test_split

_, valid = train_test_split(train, 
                            test_size=0.1,
                            stratify=train[['healthy', 'multiple_diseases', 'rust', 'scab']],
                            random_state=50)

In [5]:
import cv2
from torch.utils.data import Dataset 
import numpy as np

class ImageDataset(Dataset):

    def __init__(self, df, img_dir='./', transform=None, is_test=False):
        super().__init__() 
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
    
   
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]             
        img_path = self.img_dir + img_id + '.jpg' 
        image = cv2.imread(img_path)              
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
     
        if self.transform is not None:
            image = self.transform(image=image)['image']
        
        if self.is_test:
            return image 
        else:
         
            label = np.argmax(self.df.iloc[idx, 1:5]) 
            return image, label 

In [6]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform_train = A.Compose([
    A.Resize(450, 650),     
    A.RandomBrightnessContrast(brightness_limit=0.2, 
                               contrast_limit=0.2, p=0.3),
    A.VerticalFlip(p=0.2),    
    A.HorizontalFlip(p=0.5), 
    A.ShiftScaleRotate(       
        shift_limit=0.1,
        scale_limit=0.2,
        rotate_limit=30, p=0.3),
    A.OneOf([A.Emboss(p=1),   
             A.Sharpen(p=1),
             A.Blur(p=1)], p=0.3),
    A.PiecewiseAffine(p=0.3), 
    A.Normalize(),            
    ToTensorV2()              
])

transform_test = A.Compose([
    A.Resize(450, 650),
    A.Normalize(),      
    ToTensorV2()        
])

In [7]:
img_dir = '/kaggle/input/plant-pathology-2020-fgvc7/images/'

dataset_train = ImageDataset(train, img_dir=img_dir, transform=transform_train)
dataset_valid = ImageDataset(valid, img_dir=img_dir, transform=transform_test)

In [8]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

In [9]:
from torch.utils.data import DataLoader 

batch_size = 4

loader_train = DataLoader(dataset_train, batch_size=batch_size, 
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=g, num_workers=2)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size, 
                          shuffle=False, worker_init_fn=seed_worker,
                          generator=g, num_workers=2)

In [10]:
!pip install efficientnet-pytorch==0.7.1

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=b32a4abbd230beb2755f6e66416f6b7ce45e8e15ebf17d62b34df1d71f0e7532
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [11]:
from efficientnet_pytorch import EfficientNet 

model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=4) 

model = model.to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


  0%|          | 0.00/254M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b7


In [12]:
import torch.nn as nn 

criterion = nn.CrossEntropyLoss()

In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006, weight_decay=0.0001)

In [14]:
from transformers import get_cosine_schedule_with_warmup

epochs = 39 

scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=len(loader_train)*3, 
                                            num_training_steps=len(loader_train)*epochs)

2022-05-16 14:49:28.874278: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [15]:
from sklearn.metrics import roc_auc_score 
from tqdm.notebook import tqdm 

for epoch in range(epochs):
   
    model.train()        
    epoch_train_loss = 0 
    
    for images, labels in tqdm(loader_train):
      
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
      
        outputs = model(images)
       
        loss = criterion(outputs, labels)
        
        epoch_train_loss += loss.item() 
        loss.backward()  
        optimizer.step() 
        scheduler.step() 
        
    print(f'에폭 [{epoch+1}/{epochs}] - 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
    
    model.eval()          
    epoch_valid_loss = 0  
    preds_list = []       
    true_onehot_list = [] 
    
    with torch.no_grad(): 
      
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            
            preds = torch.softmax(outputs.cpu(), dim=1).numpy() 
         
            true_onehot = torch.eye(4)[labels].cpu().numpy() 
           
            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)
    
    print(f'에폭 [{epoch+1}/{epochs}] - 검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f} / 검증 데이터 ROC AUC : {roc_auc_score(true_onehot_list, preds_list):.4f}')  

  0%|          | 0/456 [00:00<?, ?it/s]

에폭 [1/39] - 훈련 데이터 손실값 : 1.2525
에폭 [1/39] - 검증 데이터 손실값 : 0.5673 / 검증 데이터 ROC AUC : 0.9358


  0%|          | 0/456 [00:00<?, ?it/s]

에폭 [2/39] - 훈련 데이터 손실값 : 0.5416
에폭 [2/39] - 검증 데이터 손실값 : 0.2008 / 검증 데이터 ROC AUC : 0.9869


  0%|          | 0/456 [00:00<?, ?it/s]

...

에폭 [37/39] - 훈련 데이터 손실값 : 0.0043
에폭 [37/39] - 검증 데이터 손실값 : 0.0001 / 검증 데이터 ROC AUC : 1.0000


  0%|          | 0/456 [00:00<?, ?it/s]

에폭 [38/39] - 훈련 데이터 손실값 : 0.0058
에폭 [38/39] - 검증 데이터 손실값 : 0.0001 / 검증 데이터 ROC AUC : 1.0000


  0%|          | 0/456 [00:00<?, ?it/s]

에폭 [39/39] - 훈련 데이터 손실값 : 0.0029
에폭 [39/39] - 검증 데이터 손실값 : 0.0001 / 검증 데이터 ROC AUC : 1.0000


In [16]:
dataset_test = ImageDataset(test, img_dir=img_dir, 
                            transform=transform_test, is_test=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, 
                         shuffle=False, worker_init_fn=seed_worker,
                         generator=g, num_workers=2)

dataset_TTA = ImageDataset(test, img_dir=img_dir, 
                           transform=transform_train, is_test=True)
loader_TTA = DataLoader(dataset_TTA, batch_size=batch_size, 
                        shuffle=False, worker_init_fn=seed_worker,
                        generator=g, num_workers=2)

In [17]:
model.eval() 

preds_test = np.zeros((len(test), 4)) 

with torch.no_grad():
    for i, images in enumerate(loader_test):
        images = images.to(device)
        outputs = model(images)
      
        preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
        preds_test[i*batch_size:(i+1)*batch_size] += preds_part

In [18]:
submission_test = submission.copy() 

submission_test[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds_test

In [19]:
num_TTA = 7 

preds_tta = np.zeros((len(test), 4)) 

for i in range(num_TTA):
    with torch.no_grad():
        for i, images in enumerate(loader_TTA):
            images = images.to(device)
            outputs = model(images)
         
            preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
            preds_tta[i*batch_size:(i+1)*batch_size] += preds_part

In [20]:
preds_tta /= num_TTA 

In [21]:
submission_tta = submission.copy() 

submission_tta[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds_tta

In [22]:
submission_test.to_csv('submission_test.csv', index=False)
submission_tta.to_csv('submission_tta.csv', index=False)

In [23]:
def apply_label_smoothing(df, target, alpha, threshold):

    df_target = df[target].copy()
    k = len(target) 
    
    for idx, row in df_target.iterrows():
        if (row > threshold).any():         
            row = (1 - alpha)*row + alpha/k 
            df_target.iloc[idx] = row       
    return df_target 

In [24]:
alpha = 0.001 
threshold = 0.999 

submission_test_ls = submission_test.copy()
submission_tta_ls = submission_tta.copy()

target = ['healthy', 'multiple_diseases', 'rust', 'scab'] 

submission_test_ls[target] = apply_label_smoothing(submission_test_ls, target, 
                                                   alpha, threshold)
submission_tta_ls[target] = apply_label_smoothing(submission_tta_ls, target, 
                                                  alpha, threshold)

submission_test_ls.to_csv('submission_test_ls.csv', index=False)
submission_tta_ls.to_csv('submission_tta_ls.csv', index=False)